In [1]:
import sys
import pickle
from pprint import pprint
import time
import datetime

In [2]:
from qiskit import IBMQ

In [3]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')
backend = provider.get_backend("ibmq_sydney")
device = provider.backend.ibmq_sydney

QiskitBackendNotFoundError: 'No backend matches the criteria'

In [ ]:
sys.path.append("../../libs_bell/")
from lib_graph_states import adjacency_list_for_path_graph, list_to_matrix, path_graph_state

In [ ]:
vertex_list = [20,19,16,14,11,8,5,3,2,1,4,7,10,12,15,18,21,23,24,25,26]
max_size = len(vertex_list) # largest graph size
# vertex_list += [9,16,20,30,34,44,53,55] # add extra qubits up to 65
initial_layouts = [vertex_list[:n] for n in range(1, max_size + 1)] # initial_layouts
F_list = [3 * i + 1 for i in range(max_size // 3)] # subset of focused vertices of graph size max_size
Fs = [F_list[:(n+1) // 3] for n in range(1, max_size + 1)] # subsets of focused vertices of graph size 1 to max_size
adj_lists = [adjacency_list_for_path_graph(n) for n in range(1, max_size + 1)] # adjacency list of graph size 1 to max_size
graphs = [list_to_matrix(adj_list) for adj_list in adj_lists] # adjacency matrix of graph size 1 to max_size
shots_per_circuit = 1 << 13 # shots per one execution

In [ ]:
print("max_size:", max_size)
print("shots_per_circuit:", shots_per_circuit)
print("F_list:", F_list)
print("lengths of initial_layout:", [len(i) for i in initial_layouts])
pprint(initial_layouts[:10:2])

In [ ]:
qcs = [path_graph_state(size) for size in range(1, max_size+1)]
print(len(qcs), "qcs")

In [ ]:
from lib_path_graphs import prepare_path_graph_qcs_list, execute_circuits

In [ ]:
qcs_list, nums_divide_list, nums_meas_cal_list, initial_layouts_list = \
    prepare_path_graph_qcs_list(qcs, 
                                adj_lists, 
                                backend,
                                optimization_level=3,
                                initial_layouts=[vertex_list[:qc.num_qubits] for qc in qcs])
print("the number of graph state circuits is: ", len(qcs_list))

In [ ]:
jobs = execute_circuits(qcs_list,
                        backend=backend,
                        shots=shots_per_circuit,
                        max_experiments=900,
                        optimization_level=0)
print(len(jobs), "jobs")
print("job_id:", jobs[0].job_id())

In [ ]:
dt_now = datetime.datetime.now()
job_ids_filename = "job_ids_" + dt_now.strftime('%Y%m%d_%H%M%S') + ".pkl"

In [ ]:
with open(job_ids_filename, "wb") as f:
    pickle.dump([job.job_id() for job in jobs], f)

In [ ]:
with open("properties_" + dt_now.strftime('%Y%m%d_%H%M%S') + ".pkl", "wb") as f:
    pickle.dump(device.properties(), f)

In [4]:
from qiskit.visualization import plot_circuit_layout
from qiskit import transpile
backend = provider.get_backend("ibmq_sydney")
qc_on_device = transpile(qcs[-1], backend=backend, optimization_level=1, initial_layout=vertex_list)
plot_circuit_layout(qc_on_device, backend)

QiskitBackendNotFoundError: 'No backend matches the criteria'

In [ ]:
from qiskit.visualization import plot_error_map
plot_error_map(device)